In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
!pip install tensorflow-federated

In [36]:
import os
import collections
import numpy as np
import scipy.io as sio
import tensorflow as tf
import tensorflow_federated as tff
import matplotlib.pyplot as plt
from keras import backend as K 
from keras.layers import Layer
import tensorflow_probability as tfp
tfd = tfp.distributions

################################ Functions ################################

def dc_compensation(z):
    """
    Function to estimate and remove DC impairments in the IQ branch
    Parameters:
    z: DC impaired signal sequence (numpy format)
    Returns:
    v: DC removed signal sequence
    """
    iDCest=np.mean(z[:, 0]) # estimated DC on I branch
    qDCest=np.mean(z[:, 1]) # estimated DC on I branch
    v=z-(iDCest+1j*qDCest) # remove estimated DCs
    return v

def blind_iq_compensation(z):
    """
    Function to estimate and compensate IQ impairments for the
    single-branch IQ impairment model
    Parameters:
    z: DC impaired signal sequence (numpy format)
    Returns:
    y: IQ imbalance compensated signal sequence
    """
    I=z[:, 0];Q=z[:, 1]
    theta1=(-1)*np.mean(np.sign(I)*Q)
    theta2=np.mean(abs(I)); theta3=np.mean(abs(Q))
    c1=theta1/theta2
    c2=np.sqrt((theta3**2-theta1**2)/theta2**2)
    x_u2 = I +1j*(c1*I+Q)/c2
    return x_u2


def usergen(x, y, iid, N0):
    if iid==True:
        sigma = 1 # i.i.d. user fading 
    else:
        sigma = np.random.uniform(low=0.5, high=1.5, size=None) #non-i.i.d. user fading
        
    h = np.random.rayleigh(scale=sigma, size=(x.shape[0],1))
    hIQ=np.concatenate((h,h),axis=1)
    x_u = np.multiply(hIQ,x)+ np.random.normal(0, np.sqrt(N0)/2, x.shape) # Channel-distorted noisy features
    y_u = np.real(y) # Labels
    # print(y_u)
    # y_u = np.interp(y_u, (y_u.min(), y_u.max()), (0, 1))
    D_u = tf.data.Dataset.from_tensor_slices((list(x_u),list(y_u.astype(int))))

    return D_u


def testgen(x, y, iid, N0):
    if iid==True:
        h = np.random.rayleigh(scale=1, size=(x.shape[0],1)) # i.i.d. test fading 
    else:
        h = np.random.rayleigh(scale=np.random.uniform(low=0.5, high=1.5, size=(x.shape[0],1)), size=(x.shape[0],1)) # non-i.i.d. user fading
    hIQ = np.concatenate((h,h),axis=1)
    x_test = np.multiply(hIQ,x) + np.random.normal(0, np.sqrt(N0)/2, x.shape) # Channel-distorted noisy features
    y_test = np.real(y) # Labels
    # print(y_test)
    # y_test = np.interp(y_test, (y_test.min(), y_test.max()), (0, 1))

    dataset_test = tf.data.Dataset.from_tensor_slices((list(x_test),list(y_test.astype(int))))
    
    return dataset_test


def preprocess(dataset):
    EPOCH = 5 # Number of local epochs for each aggragation round
    AGGREGATION_ROUND = 5 # Number of federated aggregation rounds
    BATCH_SIZE = 1000
    SHUFFLE_BUFFER = 20
    PREFETCH_BUFFER = 10
    def batch_format_fn(element1,element2):
        return collections.OrderedDict(
            x=tf.reshape(element1, [-1, 2]),
            y=tf.reshape(element2, [-1, 1]))
    return dataset.repeat(EPOCH).shuffle(SHUFFLE_BUFFER).batch(
        BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [37]:
class CompensateLayer(Layer): 
   def __init__(self, output_dim, **kwargs): 
      self.output_dim = output_dim 
      super(CompensateLayer, self).__init__(**kwargs) 
   def build(self, input_shape): 
    self.kernel = self.add_weight(name = 'kernel', 
      shape = (input_shape[1], self.output_dim), 
      initializer = 'normal', trainable = True) 
    super(CompensateLayer, self).build(input_shape)   
   def call(self, input_data): 
     x = K.dot(input_data, self.kernel)
     return x - K.mean(x, axis = [0, 1])
   def compute_output_shape(self, input_shape): return (input_shape[0], self.output_dim)

In [38]:
def create_keras_model():
    Q = 4 
    M = 2**Q # 16QAM modulation
    Es = 10 # Average symbol energy
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(2,)))
    model.add(CompensateLayer(2, input_shape = (2, )))
    model.add(tf.keras.layers.Dense(M, activation=tf.keras.activations.softmax))
    return model

In [39]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


In [40]:
%cd "/content/drive/MyDrive/Tasnim_BUET/11 august/fedrec_moderation"
def train(user_g, snr_g, iid_g):
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    MC = 10 # Number of Monte Carlo trials
    iid = iid_g # set to False to simulate non-iid user fading 

    ### Modulation Parameters
    Q = 4 
    M = 2**Q # 16QAM modulation
    Es = 10 # Average symbol energy

    ### Noise Parameters
    EbN0_dB = snr_g # SNR per bit (dB)
    EsN0_dB = EbN0_dB + 10*np.log10(Q) # SNR per symbol (dB)
    N0 = Es/10**(EsN0_dB/10)

    ### FedRec Training Parameters
    U = user_g # Number of wireless users taking part in federated training
    EPOCH = 5 # Number of local epochs for each aggragation round
    AGGREGATION_ROUND = 5 # Number of federated aggregation rounds
    BATCH_SIZE = 1000 
    SHUFFLE_BUFFER = 20
    PREFETCH_BUFFER = 10

    ### Load QAM I/Q Symbols
    TRAIN = sio.loadmat('TRAIN_fedrec.mat')
    Train = TRAIN['TRAIN']
    TEST = sio.loadmat('TEST_fedrec.mat')
    Test = TEST['TEST']


    ### Generate Real-valued Train/Test Features/Labels
    N_T = int(Train.shape[0]/U) # Size of local user datasets 
    x_train = np.zeros((N_T,2))
    x_train[:,0] = np.real(Train[0:N_T,0])
    x_train[:,1] = np.imag(Train[0:N_T,0])
    y_train = Train[0:N_T,1]-1

    x_test = np.zeros((Test.shape[0],2))
    x_test[:,0] = np.real(Test[:,0])
    x_test[:,1] = np.imag(Test[:,0])
    y_test = Test[:,1]-1


    example_dataset = usergen(x_train, y_train, iid, N0)
    # print(f"uplink dataset: U_U: {len(example_dataset)}")
    preprocessed_example_dataset=preprocess(example_dataset)
    example_element = next(iter((preprocessed_example_dataset)))

    def model_fn():
        keras_model = create_keras_model()
        return tff.learning.from_keras_model(
            keras_model,
            input_spec=preprocessed_example_dataset.element_spec,
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


    iterative_process = tff.learning.build_federated_averaging_process(
        model_fn,
        client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.05),
        server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

    evaluation = tff.learning.build_federated_evaluation(model_fn)
    print(evaluation)


    BER = 0
    for i in range(MC):
        
        print('##############################')
        print('Monte Carlo Trial # ', i+1)
        
        ### Generate Federated User Datasets
        federated_train_data=[]
        for u in range(U):
            D_u = usergen(x_train, y_train, iid, N0) ### Generate Local Dataset at user u
            # print(f"downlink dataset: D_U: {len(D_u)}")
            federated_train_data.append(preprocess(D_u))
            print(D_u)
                
        ### Generate Test Dataset
        test_dataset = testgen(x_test, y_test, iid, N0)
        federated_test_data=[preprocess(test_dataset)]
        print(federated_test_data)
        
        ### Federated Training
        state = iterative_process.initialize()
        for n in range(AGGREGATION_ROUND):
            state, metrics = iterative_process.next(state, federated_train_data)
            # print(str(metrics))
            
            ### Evaluate the Model
            test_metrics = evaluation(state.model, federated_test_data)
            print(str(test_metrics))
        
        BER = BER + (1-test_metrics['sparse_categorical_accuracy'])/(Q*MC)

        
    print('##############################')
    print('4QAM at Eb/N0=', EbN0_dB, 'dB')
    print('FedRec trained collaboratively by ', U, 'users')
    if iid==True:
            iidstr = 'iid' 
    else:
            iidstr = 'non-iid'
    print(iidstr, ' Rayleigh fading')
    print('BER= ', BER)

    return BER

/content/drive/MyDrive/Tasnim_BUET/11 august/fedrec_moderation


In [41]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
def main():
    ber = []
    iid = False
    dataset_setup = 'compensate in model'
    %cd "/content/drive/MyDrive/Tasnim_BUET/11 august/fedrec_moderation"
    for i in [1, 5, 10, 100, 1000]:
      print(i)
      x = train(i, 5, iid)
      ber.append(x)
      with open(f'user_change_{iid}_dataset_setup_{dataset_setup}_5db.txt', 'a+') as f:
        f.write(str(x) + '\n')
    with open(f'user_change_{iid}_dataset_setup_{dataset_setup}_5db.txt', 'a+') as f:
        f.write(str(ber) + '\n')
main()

/content/drive/MyDrive/Tasnim_BUET/11 august/fedrec_moderation
1


##############################
Monte Carlo Trial #  1
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
[<PrefetchDataset shapes: OrderedDict([(x, (None, 2)), (y, (None, 1))]), types: OrderedDict([(x, tf.float64), (y, tf.int32)])>]
OrderedDict([('sparse_categorical_accuracy', 0.728837), ('loss', 0.76562315)])
OrderedDict([('sparse_categorical_accuracy', 0.72959), ('loss', 0.76247346)])
OrderedDict([('sparse_categorical_accuracy', 0.7293668), ('loss', 0.7630901)])
OrderedDict([('sparse_categorical_accuracy', 0.7295486), ('loss', 0.7627024)])
OrderedDict([('sparse_categorical_accuracy', 0.729395), ('loss', 0.7632223)])
##############################
Monte Carlo Trial #  2
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
[<PrefetchDataset shapes: OrderedDict([(x, (None, 2)), (y, (None, 1))]), types: OrderedDict([(x, tf.float64), (y, tf.int32)])>]
OrderedDict([('sparse_categorical_accuracy', 0.7392366), ('loss', 0.7375034)])
OrderedDict([('sparse_

##############################
Monte Carlo Trial #  1
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
[<PrefetchDataset shapes: OrderedDict([(x, (None, 2)), (y, (None, 1))]), types: OrderedDict([(x, tf.float64), (y, tf.int32)])>]
OrderedDict([('sparse_categorical_accuracy', 0.4086566), ('loss', 1.6866162)])
OrderedDict([('sparse_categorical_accuracy', 0.7302426), ('loss', 0.902265)])
OrderedDict([('sparse_categorical_accuracy', 0.7386974), ('loss', 0.75825226)])
OrderedDict([('sparse_categorical_accuracy', 0.7402772), ('loss', 0.73446286)])
OrderedDict([('sparse_categorical_accuracy', 0.7402604), ('loss', 0.73284453)])
##############################
Monte Carlo Trial #  2
<TensorSlice

##############################
Monte Carlo Trial #  1
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
[<PrefetchDataset shapes: OrderedDict([(x, (None, 2)), (y, (None, 1))]), types: OrderedDict([(x, tf.float64), (y, tf.int32)])>]
OrderedDict([('sparse_categorical_accuracy', 0.387181), ('loss', 2.0891616)])
OrderedDict([('sparse_categori

##############################
Monte Carlo Trial #  1
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset shapes: ((2,), ()), types: (tf.float64, tf.int32)>
<TensorSliceDataset sha